## Setup

In [0]:
!git clone https://github.com/javieraespinosa/big-data-analytics-practicals.git

In [0]:
%cd big-data-analytics-practicals/data 
%ls -d $PWD/*
%cd ../..

In [0]:
dbutils.fs.cp("file:///databricks/driver/big-data-analytics-practicals/data/appl_stock.csv", "dbfs:/appl_stock.csv")
dbutils.fs.cp("file:///databricks/driver/big-data-analytics-practicals/data/sales_info.csv", "dbfs:/sales_info.csv")

# Les Dataframes en Pratique

## 1 Instructions

Nous jouerons dans ce TP  avec les fonctionnalités des DataFrames.

Utilisez la documentation sur l'API des DataFrames pour répondre aux questions qui vous seront posées dans ce TP.

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html


Dans ce TP, vous utiliserez le notebook pour exécuter vos programmes à partir des cellules code reconnaissables avec [] .



### Connexion à Spark

La variable "spark" est un objet de type SparkSession et c'est via cette variable que nous soumettrons des programmes distribués.

On peut vérifier notre connexion avec la commande suivante :
    

In [0]:
import pyspark
from pyspark.sql.functions import desc, col, udf
from pyspark.sql.types import StringType

from pyspark.sql import SparkSession
spark

## Exercice

Nous allons travailler sur les données en bourse du marché des pommes.

Ouvrer le fichier appl_stock.csv et chargez le dans un dataframe.

In [0]:
# Entrez vos code ici :


Afficher la structure, les types du schema

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



Afficher les dates et le prix des pommes à la clôture de la bourse qui sont inférieur à 300.

+-------------------+------------------+
|               Date|             Close|
+-------------------+------------------+
|2010-01-04 00:00:00|        214.009998|
|2010-01-05 00:00:00|        214.379993|
|2010-01-06 00:00:00|        210.969995|
|2010-01-07 00:00:00|            210.58|
|2010-01-08 00:00:00|211.98000499999998|
|2010-01-11 00:00:00|210.11000299999998|
|2010-01-12 00:00:00|        207.720001|
|2010-01-13 00:00:00|        210.650002|
|2010-01-14 00:00:00|            209.43|
|2010-01-15 00:00:00|            205.93|
|2010-01-19 00:00:00|        215.039995|
|2010-01-20 00:00:00|            211.73|
|2010-01-21 00:00:00|        208.069996|
|2010-01-22 00:00:00|            197.75|
|2010-01-25 00:00:00|        203.070002|
|2010-01-26 00:00:00|        205.940001|
|2010-01-27 00:00:00|        207.880005|
|2010-01-28 00:00:00|        199.289995|
|2010-01-29 00:00:00|        192.060003|
|2010-02-01 00:00:00|        194.729998|
+-------------------+------------------+
only showing top

Retourner les lignes sur lesquels le prix à la clotûre est < 300 et dont le prix à l'ouverture est > 300

+-------------------+------------------+----------+----------+----------+---------+------------------+
|               Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+-------------------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22 00:00:00|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28 00:00:00|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29 00:00:00|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+-------------------+------------------+----------+----------+----------+---------+------------------+



Donner les statistiques sur l'ensemble des colonnes

+-------+------------------+------------------+------------------+-----------------+-------------------+------------------+
|summary|              Open|              High|               Low|            Close|             Volume|         Adj Close|
+-------+------------------+------------------+------------------+-----------------+-------------------+------------------+
|  count|              1762|              1762|              1762|             1762|               1762|              1762|
|   mean| 313.0763111589103| 315.9112880164581| 309.8282405079457|312.9270656379113|9.422577587968218E7| 75.00174115607275|
| stddev|185.29946803981522|186.89817686485767|183.38391664371008|185.1471036170943|6.020518776592709E7| 28.57492972179906|
|    min|              90.0|         90.699997|         89.470001|        90.279999|           11475900|         24.881912|
|    max|        702.409988|        705.070023|        699.569977|       702.100021|          470249500|127.96609099999999|
+-------

Ajouter une colonne Year qui mentionnera uniquement l'année de vente .<br/>
Indice: utilisez la fonction year pour extraire l'année du champs Date.

On retournera le résultat dans un nouveau dataframe `new_df`

+-------------------+----------+----------+------------------+------------------+---------+------------------+----+
|               Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|Year|
+-------------------+----------+----------+------------------+------------------+---------+------------------+----+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|2010|
|2010-01-08 00:00:00|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|2010|
+-------------------+----------+----------+------------------+----------

## Les fonctions d'aggrégation

Déterminer le prix moyen par année de l'ensemble des colonnes :

+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|Year|         avg(Open)|         avg(High)|          avg(Low)|        avg(Close)|         avg(Volume)|    avg(Adj Close)|avg(Year)|
+----+------------------+------------------+------------------+------------------+--------------------+------------------+---------+
|2015|120.17575393253965|121.24452385714291| 118.8630954325397|120.03999980555547|  5.18378869047619E7|115.96740080555561|   2015.0|
|2013| 473.1281355634922| 477.6389272301587|468.24710264682557| 472.6348802857143|          1.016087E8| 62.61798788492063|   2013.0|
|2014| 295.1426195357143|297.56103184523823| 292.9949599801587| 295.4023416507935| 6.315273055555555E7| 87.63583323809523|   2014.0|
|2012|     576.652720788| 581.8254008040001| 569.9211606079999| 576.0497195640002|       1.319642044E8| 74.81383696800002|   2012.0|
|2016|104.50777772619044| 105.4271825436508|103.69027771825397|104.60

Déterminez  les prix maximum, minimum et moyen à la fermeture :


In [0]:
from pyspark.sql.functions import *



+----------+---------+-----------------+
| Max Close|Min Close|        Avg Close|
+----------+---------+-----------------+
|702.100021|90.279999|312.9270656379113|
+----------+---------+-----------------+



Déterminez  les prix maximum, minimum et moyen à la clôture pour chaque année :


In [0]:
import pyspark.sql.functions as f


+----------+---------+-----------------+
| Max Close|Min Close|        Avg Close|
+----------+---------+-----------------+
|702.100021|90.279999|312.9270656379113|
+----------+---------+-----------------+



## SQL

Vous analyserez à présent le fichier de vente à partir du langage SQL :


* Créer une vue sales à partir du dataframe new_df qui sera consultable en SQL :   

* Afficher la structure de la vue sales :

* Afficher les dates et le prix des pommes à la clôture de la bourse qui sont inférieur à 300$.

* Créer une vue sales300 à partir de la requête précédente qui sera consultable en SQL :   

* Déterminer  les prix maximum, minimum et moyen à la clôture pour chaque année sur la vue sales300 :


* Conserver le résultat de la requête précédente en json :

Bravo !!!

Vous avez terminé cette introduction au Dataframe, vous pouvez fermer la connexion au cluster Spark.

# Cleaning

In [0]:
dbutils.fs.rm("dbfs:/appl_stock.csv")
dbutils.fs.rm("dbfs:/sales_info.csv")